# Data Cleaning 

#### 1. Import pandas library.

In [1]:
import pandas as pd

#### 2. Import pymysql and sqlalchemy as you have learnt in the lesson of importing/exporting data. 


In [ ]:
#skip

#### 3. Create a mysql engine to set the connection to the server. Check the connection details in [this link](https://relational.fit.cvut.cz/dataset/Stats).

In [ ]:
#skip

#### 4. Import the users table.

In [2]:
users = pd.read_csv('users.csv')

#### 5. Rename Id column to userId.

In [29]:
users.rename(columns={'postId':'userId'}, inplace=True)
users.head()

,Unnamed: 0,userId,Reputation,Views,UpVotes,DownVotes
0,0,-1,1,0,5007,1920
1,1,2,101,25,3,0
2,2,3,101,22,19,0
3,3,4,101,11,0,0
4,4,5,6792,1145,662,5


#### 6. Import the posts table. 

In [8]:
posts = pd.read_csv('posts.csv')

#### 7. Rename Id column to postId and OwnerUserId to userId.

In [24]:
posts.rename(columns={'userId':'postId', 'PostId':'userId'}, inplace=True)
posts.head()


,Unnamed: 0,userId,postId,Score,ViewCount,CommentCount
0,0,1,8.0,23,1278.0,1
1,1,2,24.0,22,8198.0,1
2,2,3,18.0,54,3613.0,4
3,3,4,23.0,13,5224.0,2
4,4,5,23.0,81,NaN,3


#### 8. Define new dataframes for users and posts with the following selected columns:
**users_sliced columns**: userId, Reputation, Views, UpVotes, DownVotes  
**posts_sliced columns**: postId, Score, userId, ViewCount, CommentCount

In [31]:
column_order_users = ['userId', 'Reputation', 'Views', 'UpVotes', 'DownVotes']
users_sliced = users[column_order_users]
users_sliced


,userId,Reputation,Views,UpVotes,DownVotes
0,-1,1,0,5007,1920
1,2,101,25,3,0
2,3,101,22,19,0
3,4,101,11,0,0
4,5,6792,1145,662,5
...,...,...,...,...,...
40320,55743,1,0,0,0
40321,55744,6,1,0,0
40322,55745,101,0,0,0
40323,55746,106,1,0,0


In [33]:
column_order_posts = ['postId', 'Score', 'userId', 'ViewCount', 'CommentCount']
posts_sliced = posts[column_order_posts]
posts_sliced

,postId,Score,userId,ViewCount,CommentCount
0,8.0,23,1,1278.0,1
1,24.0,22,2,8198.0,1
2,18.0,54,3,3613.0,4
3,23.0,13,4,5224.0,2
4,23.0,81,5,NaN,3
...,...,...,...,...,...
91971,805.0,2,115374,NaN,2
91972,49365.0,0,115375,9.0,0
91973,55746.0,1,115376,5.0,2
91974,805.0,0,115377,NaN,0


#### 9. Merge the two dataframes created in the step above (8), users_sliced and posts_sliced. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [38]:
users_sliced.merge(right = posts_sliced, left_on = 'userId', right_on = 'userId')

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
0,2,101,25,3,0,24.0,22,8198.0,1
1,3,101,22,19,0,18.0,54,3613.0,4
2,4,101,11,0,0,23.0,13,5224.0,2
3,5,6792,1145,662,5,23.0,81,NaN,3
4,6,457,114,47,0,5.0,152,29229.0,5
...,...,...,...,...,...,...,...,...,...
32052,55743,1,0,0,0,24164.0,0,NaN,1
32053,55744,6,1,0,0,10961.0,1,NaN,7
32054,55745,101,0,0,0,4598.0,3,NaN,2
32055,55746,106,1,0,0,20315.0,3,59.0,0


#### 10. How many missing values do you have in your merged dataframe? On which columns?

#### 11. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before going to the next step.

#### 12. Adjust the data types in order to avoid future issues. Which ones should be changed? 